Import the librairies and modules:

In [54]:
import csv
import re
import numpy as np
from pathlib import Path
from BAG_Code_tw520.BayesianAttackGraph import parse_dot
from BAG_Code_tw520.createANDtable import create_AND_table
from BAG_Code_tw520.createORtable import create_OR_table
from BAG_Code_tw520.Tools_tree import tokenizer, Parser

from pgmpy.inference.ExactInference import BeliefPropagation
from pgmpy.factors.discrete import TabularCPD

To prepare the simulation use the following to put the graph in memory:

In [55]:
# Name of the simulation
simulation = "selection"

# Constante for probability one
ONE = 0.999

In [56]:
def to_dot(BAG, nodes, evidences, display_kts=False):
    dot = 'digraph G {\n'
    regex = r"\d+:RULE \d+ \((.*?)\):\d+\.\d+"
    print(BAG)
    prop = BeliefPropagation(BAG)
    nodes_wo_evidences = [n for n in nodes.keys() if n not in evidences.keys()]
    total_prob = prop.query(nodes_wo_evidences, evidence=evidences)
    def node_to_dot(node):
        if node not in evidences.keys():
            prob = total_prob.marginalize([n for n in nodes_wo_evidences if n != node], inplace=False).values[1]
        else:
            prob = 1
        probH = int(prob * 255)
        color = '#' + format(probH, '02X') +''+ format(255-probH, '02X') + '00'
        CVE = nodes[node]['CVE']
        LABEL = nodes[node]['label']
        number = LABEL.split(":")[0]
        prob = "{:.3f}".format(prob)
        shape = nodes[node]['shape']
        if CVE != 'null':
            return f'  {node} [label=\"{node} ({number})\\n{CVE}\\n{prob}\", color=\"{color}\", penwidth=3, shape=\"{shape}\"];\n'
        else:
            correspondance = re.search(regex, LABEL)
            if correspondance:
                return f'  {node} [label=\"{node} ({number})\\n{correspondance.group(1)}\\n{prob}\", color=\"{color}\", penwidth=3, shape=\"{shape}\"];\n'
            else:
                return f'  {node} [label="{node} ({number})\\n{LABEL}\\n{prob}", color="{color}", penwidth=3, shape="{shape}"];\n'
    for node in nodes.keys():
        dot += node_to_dot(node)
    for edge in BAG.edges():
        if display_kts or edge[0] in nodes.keys():
            dot += f'  \"{edge[0]}\" -> \"{edge[1]}\";\n'
    dot += '}'
    return dot

def kts_layer(BAG, ONE, nodes):    
    # The skills layer:
    cpd_ALskills = TabularCPD('ALskills', 2, [[1-ONE], [ONE]])
    cpd_AHskills = TabularCPD('AHskills', 2, [[ONE], [1-ONE]])
    cpd_0skills = TabularCPD('uskills', 2, [[0.5], [0.5]])
    BAG.add_edge('ALskills', 'Lskills')
    BAG.add_edge('ALskills', 'Hskills')
    BAG.add_edge('AHskills', 'Hskills')
    BAG.add_edge('Hskills', 'Lskills')
    cpd_Lskills = TabularCPD('Lskills', 2, create_OR_table([0.9, 1]).T, ['ALskills', 'Hskills'], evidence_card=2*np.ones(2))
    cpd_Hskills = TabularCPD('Hskills', 2, create_OR_table([0.1, 0.9]).T, ['ALskills', 'AHskills'], evidence_card=2*np.ones(2))


    # The knowledge layer:
    cpd_knowledge = [TabularCPD(knowledge, 2, [[0.5], [0.5]]) for knowledge in ['Known vulnerabilities', 'CQCM', 'No credentials', 'MITM', 'Permissions move', 'Privilege escalation', 'Lateral move', 'ADCS']]

    # Import all the dependencies
    cpt_l4 = create_AND_table([ONE, ONE, ONE, ONE])
    with open('./Threat_Inteligence/CVE_knowledge_tooling_skills.csv', mode='r') as ktsfile:
        reader = csv.DictReader(ktsfile)
        kts_dict = {}
        for row in reader:
            cve = row['Vulnerability']
            tmp = {'tool': row['tool'], 'skills': row['skills'], 'Type': row['Type']}
            kts_dict[cve] = tmp
    for node in nodes.items():
            id = node[0]
            node = node[1]
            if node['CVE'] != "null":
                row = kts_dict[node['CVE']]
                build_tools_tree(BAG, Parser(tokenizer(row['tool'])).parse(), id)
                BAG.add_edge(row['skills'] + 'skills', id)
                BAG.add_edge(row['Type'], id)
                parents = BAG.get_parents(id)
                BAG.add_cpds(TabularCPD(id, 2, cpt_l4.T, parents, evidence_card=2*np.ones(len(parents))))
    # We add all the necessary CPDs to the BAG
    for cpd in [cpd_0skills, cpd_Lskills, cpd_ALskills, cpd_AHskills, cpd_Hskills] + cpd_knowledge:
        if BAG.__contains__(cpd.variable):
            BAG.add_cpds(cpd)

def build_tools_tree(BAG, ast, id):
    if ast.type == 'AND':
        BAG.add_edge(ast, id)
        for child in ast.children:
            build_tools_tree(BAG, child, ast)
        if BAG.get_cpds(ast) == None:
            BAG.add_cpds(TabularCPD(ast, 2, create_AND_table([ONE, ONE]).T, ast.children, evidence_card=2*np.ones(len(ast.children))))
    elif ast.type == 'OR':
        BAG.add_edge(ast, id)
        for child in ast.children:
            build_tools_tree(BAG, child, ast)
        if BAG.get_cpds(ast) == None:
            BAG.add_cpds(TabularCPD(ast, 2, [[0.5], [0.5]]))
    else:
        BAG.add_edge(ast, id)
        if BAG.get_cpds(ast) == None:
            BAG.add_cpds(TabularCPD(ast, 2, [[0.5], [0.5]]))



In [57]:

# Path to the folder containing the tree
path = Path.cwd() / ("Personnal_simulations/output_" + simulation + "/strongly_connected_components/")
file_name = "ag-nocycles.dot"
path_to_dot = path / file_name
output_file = path / (file_name[:-4] + "_colored.dot")

# We all read from the file, adding probabilities in the same time
BAG, edges, nodes = parse_dot(open(path_to_dot, 'r').read(), ONE)

# This is the reference BAG, before the attacker has compromised any node
BAG_ref = BAG.copy()
prop0 = BeliefPropagation(BAG_ref)

# We create a dictionary to get the node number from the label
inverted_nodes = {int(v['label'].split(':')[0]): k for k, v in nodes.items()}

kts_layer(BAG, ONE, nodes)

In [58]:
evidence = {14 : 1}
with open(output_file, 'w') as f:
    f.write(to_dot(BAG, nodes, evidence, display_kts=True))
with open(path / ("BAG_ref" + ".dot"), 'w') as f:
    f.write(to_dot(BAG_ref, nodes, evidence))

BayesianNetwork with 43 nodes and 55 edges
BayesianNetwork with 19 nodes and 20 edges


## When the attacker become dcsync and we want to recalculate for domain admin
src_node = [7, 10]
dst_node = [ 14, 16, 12, 2]

In [59]:
# tab = BeliefPropagation(BAG).query(['Impacket'], evidence=evidence)
print(BAG.get_cpds('Impacket'))

ValueError: Node not present in the Directed Graph